In [1]:
# File: convert_lczero_weights.py
import gzip
import torch
import numpy as np
from model import LcZeroNet  # From previous answer

def load_lczero_weights(weights_path):
    """Load LCZero .pb.gz weights into PyTorch model"""
    model = LcZeroNet(channels=128, blocks=20)
    
    # Uncompress and parse LCZero's protobuf format
    with gzip.open(weights_path, 'rb') as f:
        data = f.read()
    
    # LCZero weight structure: [version][weights...]
    version = data[0:4]
    weights = np.frombuffer(data[4:], dtype=np.float16)
    
    # Manually map weights to PyTorch layers
    ptr = 0
    state_dict = {}
    
    # Input convolution
    state_dict['input_block.0.weight'] = torch.tensor(
        weights[ptr:ptr+3*3*112*128].reshape(128, 112, 3, 3)
    )
    ptr += 3*3*112*128
    
    # Residual blocks
    for i in range(20):
        prefix = f'res_blocks.{i}.'
        # Conv1
        state_dict[prefix+'conv1.weight'] = torch.tensor(
            weights[ptr:ptr+3*3*128*128].reshape(128, 128, 3, 3)
        )
        ptr += 3*3*128*128
        # Conv2
        state_dict[prefix+'conv2.weight'] = torch.tensor(
            weights[ptr:ptr+3*3*128*128].reshape(128, 128, 3, 3)
        )
        ptr += 3*3*128*128
    
    # Policy head
    state_dict['policy_head.0.weight'] = torch.tensor(
        weights[ptr:ptr+3*3*128*32].reshape(32, 128, 3, 3)
    )
    ptr += 3*3*128*32
    state_dict['policy_head.3.weight'] = torch.tensor(
        weights[ptr:ptr+4672*2048].reshape(4672, 2048)
    )
    ptr += 4672*2048
    
    # Value head
    state_dict['value_head.0.weight'] = torch.tensor(
        weights[ptr:ptr+3*3*128*32].reshape(32, 128, 3, 3)
    )
    ptr += 3*3*128*32
    state_dict['value_head.4.weight'] = torch.tensor(
        weights[ptr:ptr+128*256].reshape(256, 128)
    )
    ptr += 128*256
    state_dict['value_head.6.weight'] = torch.tensor(
        weights[ptr:ptr+1*256].reshape(1, 256)
    )
    
    model.load_state_dict(state_dict)
    return model

if __name__ == '__main__':
    model = load_lczero_weights('BT3-768x15x24h-swa-2790000.pb')
    torch.save(model.state_dict(), 'lczero_weights.pth')


/Users/niteshkumar/conda/envs/tf_env/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


ModuleNotFoundError: No module named 'model'